In [10]:
import pickle
import numpy as np
import pandas as pd
from sklearn.metrics.cluster import contingency_matrix 


In [11]:
%store -r test

# Take the name of the different classes (e.g. '0', '1', '2', ...)
classes = test.groupby(['label'])['label'].unique().index.to_list()

In [12]:
predicted_test = test.copy()

feature_1= 'curvature_1'


models = []
for n_class in classes: # Can be thrown an expection beacuse this is the labels of the test set and not of the models (train set)
    model = pickle.load(open('models_2/'+str(n_class)+'hmm_model.pkl', 'rb'))
    models.append(model)

#log_likelihoods = pd.DataFrame()
#log_likelihoods['predicted_class'] = classes
#log_like_max = []
#top_3_loglikelihood = []
#top_3_predicted_classes = []
predicted_class = []
#is_predicted = []
true_label=[]

for i in range(len(test)):
    angle_1 = test[feature_1].iloc[i]
    X=np.array([angle_1]).T
    true_label.append(test['label'].iloc[i])
    list_log_likelihoods = []
    for model in models:
        
        list_log_likelihoods.append(model.score(X,lengths=len(X)))
        pass

    predicted_class.append(np.argmax(list_log_likelihoods))
    

In [13]:
print(predicted_class)
print(true_label)

print(len(predicted_class))
print(len(true_label))

[8, 7, 2, 0, 3, 2, 4, 8, 5, 8, 5, 3, 2, 3, 0, 8, 0, 3, 3, 4, 4, 6, 5, 7, 7, 2, 2, 8, 3, 2, 4, 4, 3, 7, 2, 8, 7, 8, 3, 2, 5, 2, 2, 5, 8, 6, 8, 2, 4, 3, 3, 7, 7, 2, 8, 2, 0, 6, 1, 2, 8, 5, 7, 7, 7, 7, 3, 1, 5, 8, 8, 4, 0, 8, 4, 3, 4, 1, 1, 3, 3, 7, 5, 8, 7, 6, 3, 2, 2, 3, 4, 0, 1, 3, 0, 7, 6, 1, 8, 7, 4, 7, 7, 8, 2, 5, 3, 5, 2, 8, 5, 3, 5, 7, 8, 8, 8, 4, 2, 2, 8, 8, 3, 6, 6, 5, 1, 8, 3, 8, 1, 0, 8, 7, 8, 8, 3, 2, 2, 3, 8, 8, 0, 7, 5, 8, 0, 3, 1, 0, 3, 8, 5, 6, 8, 0, 5, 0, 7, 7, 7, 3, 0, 7, 6, 2, 8, 3, 3, 2, 4, 2, 6, 8, 5]
[1, 8, 2, 6, 6, 5, 1, 8, 6, 9, 8, 1, 2, 2, 0, 9, 0, 3, 9, 1, 5, 7, 6, 0, 8, 5, 3, 7, 3, 9, 3, 6, 3, 8, 3, 8, 8, 9, 0, 1, 6, 8, 6, 6, 5, 7, 3, 2, 8, 3, 2, 3, 8, 2, 2, 2, 6, 7, 7, 9, 5, 6, 8, 6, 3, 6, 7, 7, 0, 3, 9, 0, 0, 2, 0, 2, 3, 1, 1, 5, 3, 6, 0, 8, 8, 7, 9, 2, 9, 5, 9, 8, 1, 2, 0, 6, 7, 7, 9, 8, 7, 8, 8, 0, 9, 0, 3, 0, 2, 9, 6, 3, 6, 2, 5, 2, 7, 0, 9, 2, 3, 9, 6, 1, 7, 0, 7, 1, 9, 7, 7, 9, 9, 0, 5, 7, 3, 2, 3, 2, 2, 9, 6, 2, 6, 2, 0, 2, 1, 2, 3, 9, 0, 7, 9, 0, 0, 0,

In [14]:
matrix=contingency_matrix(true_label, predicted_class)

In [15]:
matrix

array([[ 7,  0,  0,  1,  3,  7,  0,  2,  1],
       [ 0,  4,  1,  1,  2,  0,  2,  0,  2],
       [ 1,  0, 10,  6,  1,  0,  0,  2,  6],
       [ 1,  0,  4, 10,  2,  0,  1,  2,  3],
       [ 0,  0,  2,  2,  1,  0,  0,  0,  4],
       [ 3,  0,  1,  3,  1,  9,  0,  5,  0],
       [ 0,  5,  0,  1,  1,  0,  7,  0,  5],
       [ 1,  0,  1,  0,  1,  1,  0, 12,  3],
       [ 1,  0,  6,  4,  1,  0,  0,  1, 11]])

In [16]:
accuracy=np.sum(np.diag(matrix))/np.sum(matrix)
print("Accuracy: ", accuracy)

Accuracy:  0.4057142857142857
